In [1]:
from allennlp.models.encoder_decoders.composed_seq2seq import ComposedSeq2Seq
from typing import Dict, Tuple

from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
 
import torch
import torch.nn as nn

from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.attention.additive_attention import AdditiveAttention
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding


from typing import Dict
import csv

from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import LabelField, TextField, Field, MetadataField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers.token import Token
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers import Tokenizer
from allennlp.data.iterators import BucketIterator, BasicIterator

from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq

import torch
import torch.nn as nn

from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.attention.additive_attention import AdditiveAttention
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.training.trainer import Trainer
from allennlp.common.util import END_SYMBOL, START_SYMBOL

import sys
sys.path.append('..')
from adat.models import get_basic_seq2seq_model
from adat.dataset import OneLangSeq2SeqReader
from adat.masker import SimpleMasker, MASK_TOKEN

In [2]:
train_path = '../../texar/examples/text_style_transfer/data/insurance_cropped/insurance.train.text'
test_path = '../../texar/examples/text_style_transfer/data/insurance_cropped/insurance.test.text'


In [3]:
masker = SimpleMasker(p=0.15)

In [4]:
reader = OneLangSeq2SeqReader(masker=masker)

In [5]:
train_dataset = reader.read(train_path)
test_dataset = reader.read(test_path)

266051it [00:27, 9780.69it/s] 
57012it [00:06, 8732.01it/s] 


In [6]:
# TODO: should skip everything but `target_tokens`
vocab = Vocabulary.from_instances(train_dataset + test_dataset)
# vocab = Vocabulary.from_files('vocab_seq2seq/')

100%|██████████| 323063/323063 [00:09<00:00, 34790.96it/s]


In [7]:
iterator = BasicIterator(batch_size=256)
iterator.index_with(vocab)

In [8]:
# next(iter(iterator(test_dataset)))

In [9]:
model = get_basic_seq2seq_model(vocab)

In [10]:
model.cuda(2)

OneLanguageSeq2SeqModel(
  (_source_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_encoder): PytorchSeq2SeqWrapper(
    (_module): LSTM(64, 32, batch_first=True)
  )
  (_attention): AdditiveAttention()
  (_target_embedder): Embedding()
  (_decoder_cell): LSTMCell(96, 32)
  (_output_projection_layer): Linear(in_features=32, out_features=2150, bias=True)
)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=test_dataset,
    patience=2,
    num_epochs=10,
    cuda_device=2
)

In [13]:
results = trainer.train()

loss: 4.4433 ||: 100%|██████████| 1040/1040 [02:31<00:00,  6.88it/s]
BLEU: 0.0000, loss: 3.9384 ||: 100%|██████████| 223/223 [03:14<00:00,  1.14it/s]
loss: 3.6067 ||: 100%|██████████| 1040/1040 [02:22<00:00,  7.29it/s]
BLEU: 0.0028, loss: 3.2103 ||: 100%|██████████| 223/223 [03:13<00:00,  1.16it/s]
loss: 2.8946 ||: 100%|██████████| 1040/1040 [02:34<00:00,  6.75it/s]
BLEU: 0.0081, loss: 2.6639 ||: 100%|██████████| 223/223 [03:22<00:00,  1.10it/s]
loss: 2.4333 ||: 100%|██████████| 1040/1040 [02:27<00:00,  7.07it/s]
BLEU: 0.0305, loss: 2.2101 ||: 100%|██████████| 223/223 [03:11<00:00,  1.17it/s]
loss: 1.9399 ||: 100%|██████████| 1040/1040 [02:12<00:00,  7.87it/s]
BLEU: 0.1295, loss: 1.6812 ||: 100%|██████████| 223/223 [03:12<00:00,  1.16it/s]
loss: 1.3989 ||: 100%|██████████| 1040/1040 [02:24<00:00,  7.18it/s]
BLEU: 0.3410, loss: 1.1655 ||: 100%|██████████| 223/223 [03:15<00:00,  1.14it/s]
loss: 0.9979 ||: 100%|██████████| 1040/1040 [02:11<00:00,  7.93it/s]
BLEU: 0.5126, loss: 0.8679 ||: 

In [16]:
results2 = trainer.train()

loss: 0.5557 ||: 100%|██████████| 1040/1040 [02:34<00:00,  6.74it/s]
BLEU: 0.6660, loss: 0.5429 ||: 100%|██████████| 223/223 [03:14<00:00,  1.14it/s]
loss: 0.5427 ||: 100%|██████████| 1040/1040 [02:19<00:00,  7.48it/s]
BLEU: 0.6694, loss: 0.5251 ||: 100%|██████████| 223/223 [03:13<00:00,  1.15it/s]
loss: 0.5223 ||: 100%|██████████| 1040/1040 [02:14<00:00,  7.71it/s]
BLEU: 0.6776, loss: 0.5083 ||: 100%|██████████| 223/223 [03:16<00:00,  1.13it/s]
loss: 0.4908 ||: 100%|██████████| 1040/1040 [02:22<00:00,  7.32it/s]
BLEU: 0.6818, loss: 0.4960 ||: 100%|██████████| 223/223 [03:12<00:00,  1.16it/s]
loss: 0.4816 ||: 100%|██████████| 1040/1040 [02:32<00:00,  6.82it/s]
BLEU: 0.6854, loss: 0.4860 ||: 100%|██████████| 223/223 [03:14<00:00,  1.15it/s]
loss: 0.4838 ||: 100%|██████████| 1040/1040 [02:35<00:00,  6.69it/s]
BLEU: 0.6888, loss: 0.4745 ||: 100%|██████████| 223/223 [03:13<00:00,  1.15it/s]
loss: 0.4682 ||: 100%|██████████| 1040/1040 [02:33<00:00,  6.78it/s]
BLEU: 0.6899, loss: 0.4678 ||: 

In [17]:
with open("model_seq2seq_masked.th", 'wb') as f:
    torch.save(model.state_dict(), f)

vocab.save_to_files("vocab_seq2seq_masked")

# Predictions

In [1]:
import sys
sys.path.append('..')

from allennlp.predictors import Seq2SeqPredictor, TextClassifierPredictor
from adat.utils import load_weights
from adat.models import get_basic_classification_model, get_basic_seq2seq_model
from allennlp.data.vocabulary import Vocabulary
from adat.masker import SimpleMasker, MASK_TOKEN

from adat.dataset import InsuranceReader, OneLangSeq2SeqReader

In [2]:
masker = SimpleMasker(p=0.15)

In [3]:
seq2seq_reader = OneLangSeq2SeqReader(masker=masker)
seq2seq_vocab = Vocabulary.from_files('vocab_seq2seq_masked')
seq2seq_model = get_basic_seq2seq_model(seq2seq_vocab)
load_weights(seq2seq_model, 'model_seq2seq_masked.th')

In [4]:
class_reader = InsuranceReader()
class_vocab = Vocabulary.from_files('vocab_classification')
class_model = get_basic_classification_model(class_vocab)
load_weights(class_model, 'model_classification.th')

In [5]:
seq2seq_predictor = Seq2SeqPredictor(seq2seq_model, seq2seq_reader)
class_predictor = TextClassifierPredictor(class_model, class_reader)

In [6]:
# ! tail {test_path} -n 30

In [7]:
sequences = """a_1213 a_877 a_1129
a_338 a_885 a_1
a_1858 a_2001 a_1860 a_1852
a_1686 a_1 a_2030 a_1737 a_1978 a_710
a_1639 a_1640
a_362 a_1979 a_1737 a_1191 a_1667 a_1656 a_1
a_1 a_583 a_552 a_552 a_1978 a_598
a_1656 a_1 a_646 a_663
a_338
a_77 a_1 a_1884 a_83 a_57 a_93 a_78 a_79 a_61 a_68 a_58
a_1978 a_1 a_1191 a_1257
a_1640
a_1 a_760 a_775 a_759 a_611 a_545
a_1978 a_789 a_778 a_771 a_797 a_793 a_762 a_765 a_769 a_796 a_791 a_786 a_2030 a_770 a_783 a_764 a_781 a_545 a_767 a_759
a_1191 a_710 a_1 a_1978
a_2113 a_87 a_58 a_83 a_1884 a_93 a_57 a_86 a_91 a_2111 a_1
a_817
a_1257 a_545 a_1138 a_1158 a_1930 a_639 a_2001 a_1111 a_1191 a_645 a_1286 a_1978 a_1937 a_1149 a_1
a_876 a_1166 a_776
a_800 a_1267 a_802
a_1 a_93 a_58 a_1884 a_68 a_57 a_61
a_604 a_785 a_786 a_876 a_794 a_775 a_338 a_781 a_579 a_777 a_778 a_793 a_765 a_1979 a_791 a_759 a_574 a_2001 a_545 a_719
a_876 a_845 a_999
a_727 a_645 a_1656 a_2001 a_791 a_621 a_809 a_545 a_794 a_1938 a_793 a_1 a_759
a_1 a_1978
a_2030 a_694 a_321 a_314 a_1 a_35 a_319 a_325 a_2052 a_1647 a_2001 a_318
a_876 a_1129
a_1785 a_645 a_723 a_1780
a_1657 a_1978 a_2190
a_1""".split('\n')

In [8]:
def predict_sequence(sequence: str, seq_to_seq_predictor: Seq2SeqPredictor = seq2seq_predictor) -> str:
    return ' '.join(seq_to_seq_predictor.predict(sequence)['predicted_tokens'])

In [9]:
for seq in sequences:
    predicted_seq = predict_sequence(seq)
    print(f'Input = {seq}\nOutput = {predicted_seq}\n')

Input = a_1213 a_877 a_1129
Output = a_1213 a_877 a_1129

Input = a_338 a_885 a_1
Output = a_338 a_885 a_1

Input = a_1858 a_2001 a_1860 a_1852
Output = a_1858 a_2001 a_1860 a_1852

Input = a_1686 a_1 a_2030 a_1737 a_1978 a_710
Output = a_2030 a_1 a_2030 a_1737 a_1978 a_710

Input = a_1639 a_1640
Output = a_1639 a_1640

Input = a_362 a_1979 a_1737 a_1191 a_1667 a_1656 a_1
Output = a_362 a_1979 a_1737 a_1191 a_1667 a_1978 a_1

Input = a_1 a_583 a_552 a_552 a_1978 a_598
Output = a_1 a_583 a_552 a_552 a_1978 a_598

Input = a_1656 a_1 a_646 a_663
Output = a_1656 a_1 a_646 a_2030

Input = a_338
Output = a_338

Input = a_77 a_1 a_1884 a_83 a_57 a_93 a_78 a_79 a_61 a_68 a_58
Output = a_77 a_1 a_1884 a_83 a_57 a_93 a_78 a_79 a_59 a_68 a_58

Input = a_1978 a_1 a_1191 a_1257
Output = a_1978 a_1 a_2002 a_1257

Input = a_1640
Output = a_1640

Input = a_1 a_760 a_775 a_759 a_611 a_545
Output = a_1 a_545 a_775 a_759 a_611 a_545

Input = a_1978 a_789 a_778 a_771 a_797 a_793 a_762 a_765 a_769 a_796 a_

In [10]:
for seq in sequences:
    masked_seq = masker.mask(seq)
    predicted_seq = predict_sequence(masked_seq)
    if MASK_TOKEN in masked_seq:
        print(f'Input = {seq}\nMasked = {masked_seq}\nOutput = {predicted_seq}\n')

Input = a_338 a_885 a_1
Masked = @MASK@ a_885 a_1
Output = a_1 a_1 a_1

Input = a_1858 a_2001 a_1860 a_1852
Masked = @MASK@ a_2001 a_1860 a_1852
Output = a_1860 a_2001 a_1860 a_1852

Input = a_1686 a_1 a_2030 a_1737 a_1978 a_710
Masked = a_1686 a_1 a_2030 a_1737 a_1978 @MASK@
Output = a_1686 a_1 a_2030 a_1737 a_1978 a_2030

Input = a_362 a_1979 a_1737 a_1191 a_1667 a_1656 a_1
Masked = a_362 a_1979 a_1737 @MASK@ a_1667 @MASK@ @MASK@
Output = a_362 a_1979 a_1 a_1 a_1667 a_1 a_1

Input = a_1 a_583 a_552 a_552 a_1978 a_598
Masked = a_1 a_583 a_552 a_552 @MASK@ a_598
Output = a_1 a_583 a_552 a_552 a_1978 a_1978

Input = a_1656 a_1 a_646 a_663
Masked = @MASK@ a_1 a_646 a_663
Output = a_1656 a_1 a_1656 a_663

Input = a_1978 a_1 a_1191 a_1257
Masked = a_1978 @MASK@ @MASK@ a_1257
Output = a_1978 a_1 a_1 a_1257

Input = a_1 a_760 a_775 a_759 a_611 a_545
Masked = a_1 a_760 a_775 @MASK@ a_611 a_545
Output = a_545 a_760 a_775 a_759 a_611 a_545

Input = a_1978 a_789 a_778 a_771 a_797 a_793 a_762 a_7

# Random pertrubations

In [11]:
import torch
from allennlp.data.iterators import BasicIterator

In [12]:
iterator = BasicIterator(batch_size=1)
iterator.index_with(seq2seq_vocab)

In [13]:
instances = [seq2seq_reader.text_to_instance(seq) for seq in sequences]

In [14]:
seq2seq_model.training = False
num_attempts = 50
should_increase_at_rate = 5.0

with torch.no_grad():
    for i, example in enumerate(iterator(instances, num_epochs=1, shuffle=False)):
        
        input_seq = sequences[i]
        true_output = seq2seq_model.forward(**example, random_perturbations=False)
        true_decoded = ' '.join(seq2seq_model.decode(true_output)['predicted_tokens'][0])
        
        if true_decoded:
            true_class_pred = class_predictor.predict(true_decoded)['probs'][1]
            class_pred = true_class_pred

            count = 0
            while class_pred / true_class_pred <= should_increase_at_rate and count <= num_attempts:
                count += 1
                output = seq2seq_model.forward(**example, random_perturbations=True)
                decoded = ' '.join(seq2seq_model.decode(output)['predicted_tokens'][0])
                class_pred = class_predictor.predict(decoded)['probs'][1]

                if class_pred / true_class_pred >= should_increase_at_rate:
                    print(f'Input          = {input_seq}\nOutput(TRUE)   = {true_decoded}\nOutput(RANDOM) = {decoded}')
                    print(f'Probability `{round(true_class_pred, 4)}` ->>> `{round(class_pred, 4)}`\n')
                    break


Input          = a_1213 a_877 a_1129
Output(TRUE)   = a_1213 a_877 a_1129
Output(RANDOM) = a_1213 a_1213 a_1213 a_2031
Probability `0.0` ->>> `0.0003`

Input          = a_338 a_885 a_1
Output(TRUE)   = a_338 a_885 a_1
Output(RANDOM) = a_2032 a_1656 a_2031 a_2031
Probability `0.0033` ->>> `0.0278`

Input          = a_1686 a_1 a_2030 a_1737 a_1978 a_710
Output(TRUE)   = a_2030 a_1 a_2030 a_1737 a_1978 a_710
Output(RANDOM) = a_340 a_340 a_2030 a_1979 a_340 a_340 a_1849
Probability `0.0056` ->>> `0.0547`

Input          = a_1639 a_1640
Output(TRUE)   = a_1639 a_1640
Output(RANDOM) = a_1639 a_1639 a_1104
Probability `0.0004` ->>> `0.0085`

Input          = a_362 a_1979 a_1737 a_1191 a_1667 a_1656 a_1
Output(TRUE)   = a_362 a_1979 a_1737 a_1191 a_1667 a_1656 a_1
Output(RANDOM) = a_362 a_1979 a_1979 a_340 a_2033 a_2032 a_340 a_1995 a_2031 a_2032
Probability `0.0198` ->>> `0.3236`

Input          = a_1 a_583 a_552 a_552 a_1978 a_598
Output(TRUE)   = a_1 a_583 a_552 a_1978 a_1978 a_598
Output(R

# Gradient test

In [10]:
from allennlp.data.iterators import BasicIterator

In [11]:
seq2seq_iterator = BasicIterator()
seq2seq_iterator.index_with(seq2seq_vocab)

In [12]:
class_iterator = BasicIterator()
class_iterator.index_with(class_vocab)

In [13]:
seq2seq_reader.text_to_instance('a_1978 a_645')

In [ ]:
seq2seq_model()